In [3]:
import transformers

model_name = "DeepPavlov/rubert-base-cased"
model = transformers.AutoModel.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
import pandas as pd

data = pd.read_csv("test50.csv")

In [7]:
from tqdm import tqdm
Embedding = []
Target = []
for i in tqdm(range(len(data))):
  sentence = data["Отзывы"][i]
  tokenized = tokenizer(sentence, return_tensors="pt")
  embeddings = model(**tokenized, output_hidden_states=True).last_hidden_state
  Embedding.append(embeddings[0][1])
  del embeddings
  if data["разметка"][i][0] == "+":
    Target.append(1)
  elif data["разметка"][i][0] == "-":
    Target.append(0)

100%|██████████| 50/50 [00:09<00:00,  5.21it/s]


In [8]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, features, target):
        super().__init__()
        self.features = features
        self.target = target

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        X = self.features[idx]
        y = self.target[idx]

        return X, y


test_dataset =CustomDataset(Embedding, Target)

In [9]:
import time
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader,random_split
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.models.vision_transformer import MLPBlock

### Training function
def train_epoch(model, device, dataloader, loss_fn, optimizer):
    model.train()
    train_loss = []
    accuracy=0
    for embed, y in dataloader:
        embed = embed.detach().to(device)
        y = y.to(device)
        # Embedded data
        pred = model(embed)
        # Accuracy
        ps = torch.exp(pred)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == y.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor))
        # Loss function
        loss = loss_fn(pred, y)
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.append(loss.detach().cpu().numpy())
    return np.mean(train_loss), accuracy/len(dataloader)

### Testing function
def test_epoch(model, device, dataloader, loss_fn):
    model.eval()
    with torch.no_grad(): # No need to track the gradients
        # Define the lists to store the outputs for each batch
        conc_out = []
        conc_label = []
        accuracy=0
        for embed, y in dataloader:
            # Move tensor to the proper device
            embed = embed.to(device)
            y = y.to(device)
            # Decode data
            pred = model(embed)
            # Accuracy
            ps = torch.exp(pred)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == y.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor))
            conc_out.append(pred.cpu())
            conc_label.append(y.cpu())
        # Create a single tensor with all the values in the lists
        conc_out = torch.cat(conc_out)
        conc_label = torch.cat(conc_label)
        # Evaluate global loss
        val_loss = loss_fn(conc_out, conc_label)
    return val_loss.data, accuracy/len(dataloader)

In [11]:
!pip install qiskit
!pip install qiskit_machine_learning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 kB 10.0 MB/s eta 0:00:00
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp310-cp310-linux_x86_64.whl size=512550 sha256=e13ef9d65622acbc9bb47f0a0b303d85500e57be8005f8fb4244cb90e53a6d93
  Stored in directory: /root/.cache/pip/wheels/73/c8/f7/c25448dab74c3acf4848bc25d513c736bb93910277e1528ef4
Successfully built fastdtw


In [12]:
import numpy as np
import matplotlib.pyplot as plt

from torch import Tensor
from torch.nn import Linear, CrossEntropyLoss, MSELoss
from torch.optim import LBFGS

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_machine_learning.utils import algorithm_globals
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
import torch
from torch import cat, no_grad, manual_seed
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim
from torch.nn import (
    Module,
    Conv2d,
    Linear,
    Dropout2d,
    NLLLoss,
    MaxPool2d,
    Flatten,
    Sequential,
    ReLU,
)
import torch.nn.functional as F

In [13]:
def create_qnn(qbit = 2, reps=1):
    feature_map = ZZFeatureMap(qbit)
    ansatz = RealAmplitudes(qbit, reps=reps)
    qc = QuantumCircuit(qbit)
    qc.compose(feature_map, inplace=True)
    qc.compose(ansatz, inplace=True)

    qnn = EstimatorQNN(
        circuit=qc,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        input_gradients=True,
    )
    return qnn

In [14]:


class Net(Module):
    def __init__(self, qbit = 2, reps=1):
        super().__init__()

        self.fc1 = Linear(768, 400)
        self.fc2 = Linear(400, 100)
        self.fc3 = Linear(100,qbit)  # 2-dimensional input to QNN
        self.qnn = TorchConnector(create_qnn(qbit, reps))  # Apply torch connector, weights chosen
        # uniformly at random from interval [-1,1].
        self.fc4 = Linear(1, 1)  # 1-dimensional output from QNN

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.qnn(x)  # apply QNN
        x = self.fc4(x)
        return cat((x, 1 - x), -1)

In [15]:

model = Net(qbit = 3, reps=2)

model.load_state_dict(torch.load("q_model_3qubits.pt"))

<ipython-input-13-8d525dc1955b>:8: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(
<ipython-input-13-8d525dc1955b>:8: UserWarning: No number of qubits was not specified (None) and was retrieved from `circuit` (3). If `circuit` is transpiled, this may cause unstable behaviour.
  qnn = EstimatorQNN(
<ipython-input-15-37da96de0134>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed 

<All keys matched successfully>

In [28]:
test_dataset =CustomDataset(Embedding, Target)
batch_size = 50
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,shuffle=True)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
loss_fn = nn.NLLLoss()
test_loss, test_acc = test_epoch(model, device, test_loader, loss_fn)
print('Accuracy:', test_acc)

Accuracy: tensor(0.8200)
